# Agnostic Person Generation

This notebook provides a utility to generate "Agnostic Person" images and parsing maps. 
An agnostic person is a representation where the original garment and arms are removed (replaced by a gray area). 
The gray area is slightly larger than the original garment to break the **Cloth Shape Bias**, ensuring the flow renderer doesn't confine the new garment to the mask of the old one.

In [ ]:
import os
import cv2
import numpy as np
from PIL import Image
from pathlib import Path
import matplotlib.pyplot as plt

# LIP Labels Mapping
# Labels to MASK OUT (Gray area):
# 5: Upper-clothes, 6: Dress, 7: Coat, 14: Left-arm, 15: Right-arm
AGNOSTIC_LABELS = [5, 6, 7, 14, 15]

# Labels to PRESERVE (Keep original pixels even if mask overlaps):
# 1: Hat, 2: Hair, 4: Sunglasses, 13: Face
PRESERVE_LABELS = [1, 2, 4, 13]

## Agnostic Generation Function

In [ ]:
def get_agnostic_person(img_path, parse_path, dilation_kernel_size=25):
    """
    Generates agnostic image and parsing map with feature preservation.
    """
    # Load image and parsing
    image = np.array(Image.open(Path(img_path)).convert("RGB"))
    parse = np.array(Image.open(Path(parse_path)))

    # 1. Create preservation mask (Parts of the person we want to keep at all costs)
    preserve_mask = np.isin(parse, PRESERVE_LABELS).astype(np.uint8)

    # 2. Create binary mask for agnostic areas (Clothes and Arms)
    agnostic_mask = np.isin(parse, AGNOSTIC_LABELS).astype(np.uint8)

    # 3. Dilate the agnostic mask to break 'Cloth Shape Bias'
    kernel = np.ones((dilation_kernel_size, dilation_kernel_size), np.uint8)
    dilated_mask = cv2.dilate(agnostic_mask, kernel, iterations=1)

    # 4. Refine Dilated Mask: Subtract the preservation mask
    # This ensures gray area doesn't cover hair, face, or hat
    final_agnostic_mask = (dilated_mask == 1) & (preserve_mask == 0)

    # Create Agnostic Image
    agnostic_img = image.copy()
    # Replace masked area with neutral gray (128, 128, 128)
    agnostic_img[final_agnostic_mask] = [128, 128, 128]
    
    # Create Agnostic Parsing
    # We clear the garment/arm labels from the parsing map
    agnostic_parse = parse.copy()
    agnostic_parse[agnostic_mask == 1] = 0 # Set to background

    return Image.fromarray(agnostic_img), Image.fromarray(agnostic_parse)

## Target Guidance Map Generation

To achieve true garment independence, we provide a **Guidance Map** to the FEM (Flow Estimation Module).
This map has label `5` (Upper-clothes) in a smooth, neutral torso region, so the network estimates flow based on the *pose* rather than the *original garment's silhouette*.

In [ ]:
def get_target_guidance(parse_path, iterations=5):
    """
    Generates a neutral guidance parsing map for the FEM.
    """
    parse = np.array(Image.open(Path(parse_path)))
    
    # 1. Isolate garment area (Upper-clothes, Dress, Coat)
    garment_mask = np.isin(parse, [5, 6, 7]).astype(np.uint8)
    
    # 2. Smooth/Morph the mask to remove 'bias' (wrinkles, specific edges)
    kernel = np.ones((5,5), np.uint8)
    # Closing fills small holes and smooths edges
    neutral_mask = cv2.morphologyEx(garment_mask, cv2.MORPH_CLOSE, kernel, iterations=iterations)
    # Dilation ensures it fully covers the torso guidance area for the FEM
    neutral_mask = cv2.dilate(neutral_mask, kernel, iterations=2)
    
    # 3. Create a Guidance Parsing Map
    # We copy the original but replace the garment area with our 'neutral' one
    guidance_parse = parse.copy()
    # Set background for the garment area first
    guidance_parse[np.isin(parse, [5, 6, 7])] = 0 
    # Apply the neutral garment shape as label 5
    guidance_parse[neutral_mask == 1] = 5
    
    return Image.fromarray(guidance_parse)

## Visualization / Test

In [ ]:
person_id = "00005_00.jpg"
current_dir = Path.cwd()

# Auto-detect project root
if (current_dir / "data").exists():
    project_root = current_dir
else:
    project_root = current_dir.parent

img_path = project_root / "data" / "viton-hd" / "train" / "image" / person_id
parse_path = project_root / "data" / "viton-hd" / "train" / "image-parse-v3" / person_id.replace('.jpg', '.png')

if img_path.exists() and parse_path.exists():
    agn_img, agn_prse = get_agnostic_person(img_path, parse_path)
    guidance_prse = get_target_guidance(parse_path)
    
    # Visualize results
    fig, axes = plt.subplots(1, 3, figsize=(20, 10))
    axes[0].imshow(Image.open(img_path))
    axes[0].set_title("Original Person")
    axes[0].axis('off')
    
    axes[1].imshow(agn_img)
    axes[1].set_title("Agnostic Person (Stage 3 Input)")
    axes[1].axis('off')
    
    axes[2].imshow(guidance_prse)
    axes[2].set_title("Guidance Parsing (FEM Stage 2 Input)")
    axes[2].axis('off')
    
    plt.tight_layout()
    plt.show()
else:
    print(f"Test paths not found. Please check paths: \n{img_path}\n{parse_path}")